In [ ]:
import pandas as pd
import requests
import io

# Обработка

Соберем все нужные сезоны, полная статистика на fbref есть начиная с сезона 17/18, значит нам нужны трансферы с сезона 18/19 (чтобы была статистика по сезону до перехода) То есть сезоны 18/19, 19/20 и 20/21 (можно убрать 20/21).


In [ ]:
years = ["2018", "2019", "2020"]
comps = ["english_premier_league", "german_bundesliga_1", "italian_serie_a", "french_ligue_1", "spanish_primera_division"]
frames = {}
season_names = []
for year in years:
  for comp in comps:
    url = "https://raw.githubusercontent.com/ewenme/transfers/master/data/" + year + "/" + comp + ".csv"
    download = requests.get(url).content
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    df = df[df['transfer_movement'] == 'in']
    #df = df[df['fee'] != 'Loan']
    #df = df[df['fee'].str.match('^E.*') == False]
    name = comp[:2] + "_" + year[2:]
    frames[name] = df
    season_names.append(name)

Теперь нужно оставить только трансферы из команд выступавших в прошлом сезоне (для зимних - в этом) в отдной из топ 5 лиг. Составим для каждого сезона список команд игравших в топ 5 лигах.

In [ ]:
teams = {}
club_names = []
for comp in comps:
  url = "https://raw.githubusercontent.com/ewenme/transfers/master/data/" + "2017" + "/" + comp + ".csv"
  download = requests.get(url).content
  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
  club_names += df['club_name'].unique().tolist()

teams["2017"] = club_names

for year in years:
  club_names = []
  for comp in comps:
    name = comp[:2] + "_" + year[2:]
    club_names += frames[name]['club_name'].unique().tolist()

  teams[year] = club_names

In [ ]:
teams['2017']

['Chelsea FC',
 'Tottenham Hotspur',
 'Manchester City',
 'Liverpool FC',
 'Arsenal FC',
 'Manchester United',
 'Everton FC',
 'Southampton FC',
 'AFC Bournemouth',
 'West Bromwich Albion',
 'West Ham United',
 'Leicester City',
 'Stoke City',
 'Crystal Palace',
 'Swansea City',
 'Burnley FC',
 'Watford FC',
 'Newcastle United',
 'Brighton & Hove Albion',
 'Huddersfield Town',
 'Bayern Munich',
 'RB Leipzig',
 'Borussia Dortmund',
 'TSG 1899 Hoffenheim',
 '1. FC Köln',
 'Hertha BSC',
 'SC Freiburg',
 'SV Werder Bremen',
 'Borussia Mönchengladbach',
 'FC Schalke 04',
 'Eintracht Frankfurt',
 'Bayer 04 Leverkusen',
 'FC Augsburg',
 'Hamburger SV',
 '1.FSV Mainz 05',
 'VfL Wolfsburg',
 'VfB Stuttgart',
 'Hannover 96',
 'Atalanta BC',
 'Benevento Calcio',
 'Bologna FC 1909',
 'Cagliari Calcio',
 'Chievo Verona',
 'FC Crotone',
 'ACF Fiorentina',
 'Genoa CFC',
 'Hellas Verona',
 'Inter Milan',
 'Juventus FC',
 'SS Lazio',
 'AC Milan',
 'SSC Napoli',
 'AS Roma',
 'UC Sampdoria',
 'US Sassuol

В колонке 'club_involved_name' названия команд записаны в сокращенном виде, поэтому составим словарь сокращений с помощью списка исходящих трансферов.
(Можно было бы изначально просто пересечь списки трансферов на вход и на выход, но в таком случае у нас бы были трансферы из команд игравших прошлый сезон во второй лиге и не было бы трансферов из вылетевших команд)

Построим датафрейм исходящих трансферов

In [ ]:
frames_out = {}
for year in years:
  for comp in comps:
    url = "https://raw.githubusercontent.com/ewenme/transfers/master/data/" + year + "/" + comp + ".csv"
    download = requests.get(url).content
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    df = df[df['transfer_movement'] == 'out']
    df = df[df['fee'] != 'Loan']
    df = df[df['fee'].str.match('^E.*') == False]
    name = comp[:2] + "_" + year[2:]
    frames_out[name] = df

Объеденим датафреймы и построим словарь, сначала на примере одного сезона премьер лиги.

In [ ]:
pd.merge(frames['en_18'], frames_out['en_18'], how='inner', left_on='player_name', right_on='player_name')

,club_name_x,player_name,age_x,position_x,club_involved_name_x,fee_x,transfer_movement_x,transfer_period_x,fee_cleaned_x,league_name_x,year_x,season_x,club_name_y,age_y,position_y,club_involved_name_y,fee_y,transfer_movement_y,transfer_period_y,fee_cleaned_y,league_name_y,year_y,season_y
0,Manchester City,Riyad Mahrez,27,Right Winger,Leicester,£61.02m,in,Summer,61.02,Premier League,2018,2018/2019,Leicester City,27,Right Winger,Man City,£61.02m,out,Summer,61.020,Premier League,2018,2018/2019
1,Manchester City,Pablo Maffeo,20,Right-Back,Girona,"End of loanJun 30, 2018",in,Summer,0.00,Premier League,2018,2018/2019,Manchester City,20,Right-Back,VfB Stuttgart,£8.10m,out,Summer,8.100,Premier League,2018,2018/2019
2,Manchester City,Jason Denayer,23,Centre-Back,Galatasaray,"End of loanJun 30, 2018",in,Summer,0.00,Premier League,2018,2018/2019,Manchester City,23,Centre-Back,Olympique Lyon,£9.00m,out,Summer,9.000,Premier League,2018,2018/2019
3,Manchester City,Olarenwaju Kayode,25,Centre-Forward,Shakhtar D.,"End of loanJun 30, 2018",in,Summer,0.00,Premier League,2018,2018/2019,Manchester City,25,Centre-Forward,Shakhtar D.,£2.70m,out,Summer,2.700,Premier League,2018,2018/2019
4,Liverpool FC,Harry Wilson,21,Right Winger,Liverpool U23,-,in,Summer,0.00,Premier League,2018,2018/2019,Liverpool FC,21,Right Winger,Derby,Loan fee:£990Th.,out,Summer,0.990,Premier League,2018,2018/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Southampton FC,Ryan Seager,22,Centre-Forward,Telstar,"End of loanJan 7, 2019",in,Winter,0.00,Premier League,2018,2018/2019,Southampton FC,22,Centre-Forward,Yeovil Town,Free transfer,out,Winter,0.000,Premier League,2018,2018/2019
74,Wolverhampton Wanderers,Aaron Collins,21,Centre-Forward,Colchester Utd.,"End of loanJan 2, 2019",in,Winter,0.00,Premier League,2018,2018/2019,Wolverhampton Wanderers,21,Centre-Forward,Without Club,-,out,Winter,0.000,Premier League,2018,2018/2019
75,Wolverhampton Wanderers,Ben Stevenson,21,Central Midfield,Jumilla,"End of loanJan 19, 2019",in,Winter,0.00,Premier League,2018,2018/2019,Wolverhampton Wanderers,21,Central Midfield,Colchester Utd.,£225Th.,out,Winter,0.225,Premier League,2018,2018/2019
76,Wolverhampton Wanderers,Danny Batth,28,Centre-Back,Middlesbrough,"End of loanJan 28, 2019",in,Winter,0.00,Premier League,2018,2018/2019,Wolverhampton Wanderers,28,Centre-Back,Stoke City,£3.11m,out,Winter,3.110,Premier League,2018,2018/2019


Видим, что у некоторых игроков было сразу 2 трансфера, поэтому получаются плохие пары, потребуем что бы цена совпадала.

In [ ]:
test = pd.merge(frames['en_18'], frames_out['en_18'], how='inner', left_on='player_name', right_on='player_name')
test = test[test['fee_x'] == test['fee_y']]

In [ ]:
test_dict = {**pd.Series(test.club_involved_name_y.values,index=test.club_name_x).to_dict(), **pd.Series(test.club_involved_name_x.values,index=test.club_name_y).to_dict()}
print(test_dict)

{'Manchester City': 'Man City', 'Fulham FC': 'Fulham', 'Chelsea FC': 'Chelsea', 'Everton FC': 'Everton', 'Crystal Palace': 'Crystal Palace', 'West Ham United': 'West Ham', 'Burnley FC': 'Burnley', 'Leicester City': 'Leicester', 'Wolverhampton Wanderers': 'Wolves', 'Watford FC': 'Watford', 'Brighton & Hove Albion': 'Brighton', 'Huddersfield Town': 'Huddersfield', 'Southampton FC': 'Southampton', 'AFC Bournemouth': 'Bournemouth', 'Liverpool FC': 'Liverpool', 'Arsenal FC': 'Arsenal', 'Newcastle United': 'Newcastle'}


Теперь построим общий словарь. Объеденим датафреймы разных лиг одного сезона в один, что бы охватить больше трансферов. (Можно объеденить и сезоны тоже, но тогда будет много игроков с несколькими трансферами). Наверно можно автоматизировать, но я не смог.

In [ ]:
year_18 = pd.concat([frames['en_18'], frames['fr_18'], frames['it_18'], frames['ge_18'], frames['sp_18']])
year_19 = pd.concat([frames['en_19'], frames['fr_19'], frames['it_19'], frames['ge_19'], frames['sp_19']])
year_20 = pd.concat([frames['en_20'], frames['fr_20'], frames['it_20'], frames['ge_20'], frames['sp_20']])

year_18_out = pd.concat([frames_out['en_18'], frames_out['fr_18'], frames_out['it_18'], frames_out['ge_18'], frames_out['sp_18']])
year_19_out = pd.concat([frames_out['en_19'], frames_out['fr_19'], frames_out['it_19'], frames_out['ge_19'], frames_out['sp_19']])
year_20_out = pd.concat([frames_out['en_20'], frames_out['fr_20'], frames_out['it_20'], frames_out['ge_20'], frames_out['sp_20']])

Составим словари для каждого сезона, пришлось убрать бесплатные трансферы из - за них случались ошибки.

In [ ]:
dict_18 = pd.merge(year_18, year_18_out, how='inner', left_on='player_name', right_on='player_name')
dict_18 = dict_18[dict_18['fee_x'] == dict_18['fee_y']]
dict_18 = dict_18[dict_18['transfer_period_x'] == dict_18['transfer_period_y']]
dict_18 = dict_18[dict_18['fee_cleaned_x'] != 0.0]
dict_18 = {**pd.Series(dict_18.club_name_x.values,index=dict_18.club_involved_name_y).to_dict(), **pd.Series(dict_18.club_name_y.values,index=dict_18.club_involved_name_x).to_dict()}

dict_19 = pd.merge(year_19, year_19_out, how='inner', left_on='player_name', right_on='player_name')
dict_19 = dict_19[dict_19['fee_x'] == dict_19['fee_y']]
dict_19 = dict_19[dict_19['transfer_period_x'] == dict_19['transfer_period_y']]
dict_19 = dict_19[dict_19['fee_cleaned_x'] != 0.0]
dict_19 = {**pd.Series(dict_19.club_name_x.values,index=dict_19.club_involved_name_y).to_dict(), **pd.Series(dict_19.club_name_y.values,index=dict_19.club_involved_name_x).to_dict()}

dict_20 = pd.merge(year_20, year_20_out, how='inner', left_on='player_name', right_on='player_name')
dict_20 = dict_20[dict_20['fee_x'] == dict_20['fee_y']]
dict_20 = dict_20[dict_20['transfer_period_x'] == dict_20['transfer_period_y']]
dict_20 = dict_20[dict_20['fee_cleaned_x'] != 0.0]
dict_20 = {**pd.Series(dict_20.club_name_x.values,index=dict_20.club_involved_name_y).to_dict(), **pd.Series(dict_20.club_name_y.values,index=dict_20.club_involved_name_x).to_dict()}

In [ ]:
dict_teams = {**dict_18, **dict_19, **dict_20}
dict_teams

{'1. FC Köln': '1. FC Köln',
 '1.FC Nuremberg': '1.FC Nuremberg',
 '1.FSV Mainz 05': '1.FSV Mainz 05',
 'AC Milan': 'AC Milan',
 'AS Roma': 'AS Roma',
 'Ajax': 'Sevilla FC',
 'Alavés': 'Deportivo Alavés',
 'Amiens SC': 'Amiens SC',
 'Arm. Bielefeld': 'Arminia Bielefeld',
 'Arsenal': 'Arsenal FC',
 'Aston Villa': 'Aston Villa',
 'Atalanta': 'Atalanta BC',
 'Atalanta BC': 'Atalanta BC',
 'Athletic': 'Athletic Bilbao',
 'Atlético Madrid': 'Atlético Madrid',
 'Basaksehir': 'Spezia Calcio',
 'Bay. Leverkusen': 'Bayer 04 Leverkusen',
 'Bayern Munich': 'Bayern Munich',
 'Benevento': 'Benevento Calcio',
 'Bologna': 'Bologna FC 1909',
 'Bor. Dortmund': 'Borussia Dortmund',
 "Bor. M'gladbach": 'Borussia Mönchengladbach',
 'Bournemouth': 'AFC Bournemouth',
 'Brescia': 'Brescia Calcio',
 'Brighton': 'Brighton & Hove Albion',
 'Burnley': 'Burnley FC',
 'CA Osasuna': 'CA Osasuna',
 'CD Leganés': 'CD Leganés',
 'Cagliari Calcio': 'Cagliari Calcio',
 'Cardiff': 'Cardiff City',
 'Celta de Vigo': 'Celta

В словарь ошибочно попали Аякс и Перуджа, удалим их из словаря. Кажется, больше ошибок нет.

In [ ]:
dict_teams.pop('Ajax')
dict_teams.pop('Perugia')
dict_teams.pop('Basaksehir')

'Spezia Calcio'

Теперь заменим в датафреймах всех лиг все названия на полные.

In [ ]:
for i in season_names:
  frames[i].replace(to_replace = dict_teams, inplace = True)

Оставляем трансферы только из топ 5 лиг.

In [ ]:
for year in years:
  for comp in comps:
    name = comp[:2] + "_" + year[2:]
    df = frames[name]
    pr_year = str(int(year) - 1)
    df = df[df['club_involved_name'].isin(teams[pr_year])]
    frames[name] = df 

In [ ]:
frames['sp_20']

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,Real Madrid,Martin Ødegaard,21,Attacking Midfield,Real Sociedad,"End of loanJul 20, 2020",in,Summer,0.0,Primera Division,2020,2020/2021
1,Real Madrid,Takefusa Kubo,19,Right Winger,RCD Mallorca,"End of loanJul 20, 2020",in,Summer,0.0,Primera Division,2020,2020/2021
2,Real Madrid,Sergio Reguilón,23,Left-Back,Sevilla FC,"End of loanAug 22, 2020",in,Summer,0.0,Primera Division,2020,2020/2021
3,Real Madrid,Achraf Hakimi,21,Right Midfield,Borussia Dortmund,"End of loanJun 30, 2020",in,Summer,0.0,Primera Division,2020,2020/2021
4,Real Madrid,Óscar Rodríguez,22,Attacking Midfield,CD Leganés,"End of loanJul 20, 2020",in,Summer,0.0,Primera Division,2020,2020/2021
...,...,...,...,...,...,...,...,...,...,...,...,...
674,Deportivo Alavés,Facundo Pellistri,19,Right Winger,Manchester United,loan transfer,in,Winter,NaN,Primera Division,2020,2020/2021
675,Deportivo Alavés,Iñigo Córdoba,23,Left Winger,Athletic Bilbao,loan transfer,in,Winter,NaN,Primera Division,2020,2020/2021
678,SD Huesca,Denis Vavro,24,Centre-Back,SS Lazio,loan transfer,in,Winter,NaN,Primera Division,2020,2020/2021
683,Celta de Vigo,Aarón Martín,23,Left-Back,1.FSV Mainz 05,loan transfer,in,Winter,NaN,Primera Division,2020,2020/2021


# Разбиение на летние/зимние окна и отсечение игроков игравших в прошлом клубе меньше года

Вероятно нам нужно будет, чтобы у анализируемого игрока была статистика как минимум за последний год (в противном случае можно пользоваться таблицей выше). Значит нужно проверить, что в прошлое трансферное окно игрок не менял клуб.

Сначала разделим получившиеся трансферы на летние и зимние.

In [ ]:
frames2 = {}
for year in years:
  for comp in comps:
    name = comp[:2] + "_" + year[2:] 
    df = frames[name]
    summer = df[df['transfer_period'] == 'Summer']
    winter = df[df['transfer_period'] == 'Winter']
    name_s = comp[:2] + "_" + year[2:] + "_sum"
    name_w = comp[:2] + "_" + year[2:] + "_win"
    frames2[name_s] = summer
    frames2[name_w] = winter

Теперь нужно потребовать что бы в прошлое трансферное окно игрок никуда не переходил включая аренды. Заново скачаем переходы по всем окнам с зимы 17/18 по лето 20/21.

In [ ]:
years_tmp = ["2017", "2018", "2019", "2020"]
frames_tmp = {}
for year in years_tmp:
  for comp in comps:
    url = "https://raw.githubusercontent.com/ewenme/transfers/master/data/" + year + "/" + comp + ".csv"
    download = requests.get(url).content
    df = pd.read_csv(io.StringIO(download.decode('utf-8')))
    df = df[df['transfer_movement'] == 'in']
    name_s = comp[:2] + "_" + year[2:] + "_sum"
    name_w = comp[:2] + "_" + year[2:] + "_win"
    if (year != "2017"):
      frames_tmp[name_s] = df[df['transfer_period'] == 'Summer']
    if (year != "2020"):
      frames_tmp[name_w] = df[df['transfer_period'] == 'Winter']

Функция для получения названия предыдущего трансферного окна:

In [ ]:
def prev_window(window):
  league = window[:2]
  year = window[3:5]
  season = window[6:]
  if (season == "sum"):
    pr_window = league + "_" + str(int(year) - 1) + "_win"
    return pr_window
  else:
    pr_window = league + "_" + year + "_sum"
    return pr_window 

Теперь для каждого игрока из очищенного датафрейма проверим, что он никуда не переходил в прошлое окно.

In [ ]:
frames2['en_18_sum']

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,Manchester City,Riyad Mahrez,27,Right Winger,Leicester City,£61.02m,in,Summer,61.02,Premier League,2018,2018/2019
3,Manchester City,Tosin Adarabioyo,21,Centre-Back,West Bromwich Albion,"End of loanMay 31, 2019",in,Summer,0.00,Premier League,2018,2018/2019
9,Manchester City,Pablo Maffeo,20,Right-Back,Girona FC,"End of loanJun 30, 2018",in,Summer,0.00,Premier League,2018,2018/2019
31,Manchester United,Lee Grant,35,Goalkeeper,Stoke City,£1.53m,in,Summer,1.53,Premier League,2018,2018/2019
36,Manchester United,Andreas Pereira,22,Attacking Midfield,Valencia CF,"End of loanJun 30, 2018",in,Summer,0.00,Premier League,2018,2018/2019
...,...,...,...,...,...,...,...,...,...,...,...,...
525,Fulham FC,Calum Chambers,23,Centre-Back,Arsenal FC,Loan,in,Summer,0.00,Premier League,2018,2018/2019
526,Fulham FC,Sergio Rico,24,Goalkeeper,Sevilla FC,Loan,in,Summer,0.00,Premier League,2018,2018/2019
527,Fulham FC,Luciano Vietto,24,Centre-Forward,Atlético Madrid,Loan,in,Summer,0.00,Premier League,2018,2018/2019
528,Fulham FC,Timothy Fosu-Mensah,20,Right-Back,Manchester United,Loan,in,Summer,0.00,Premier League,2018,2018/2019


In [ ]:
for window in frames2.keys():
  frames2[window] = frames2[window][~frames2[window]['player_name'].isin(frames_tmp[prev_window(window)]['player_name'])]

In [ ]:
frames2['en_18_sum']

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
0,Manchester City,Riyad Mahrez,27,Right Winger,Leicester City,£61.02m,in,Summer,61.02,Premier League,2018,2018/2019
3,Manchester City,Tosin Adarabioyo,21,Centre-Back,West Bromwich Albion,"End of loanMay 31, 2019",in,Summer,0.00,Premier League,2018,2018/2019
9,Manchester City,Pablo Maffeo,20,Right-Back,Girona FC,"End of loanJun 30, 2018",in,Summer,0.00,Premier League,2018,2018/2019
31,Manchester United,Lee Grant,35,Goalkeeper,Stoke City,£1.53m,in,Summer,1.53,Premier League,2018,2018/2019
36,Manchester United,Andreas Pereira,22,Attacking Midfield,Valencia CF,"End of loanJun 30, 2018",in,Summer,0.00,Premier League,2018,2018/2019
...,...,...,...,...,...,...,...,...,...,...,...,...
525,Fulham FC,Calum Chambers,23,Centre-Back,Arsenal FC,Loan,in,Summer,0.00,Premier League,2018,2018/2019
526,Fulham FC,Sergio Rico,24,Goalkeeper,Sevilla FC,Loan,in,Summer,0.00,Premier League,2018,2018/2019
527,Fulham FC,Luciano Vietto,24,Centre-Forward,Atlético Madrid,Loan,in,Summer,0.00,Premier League,2018,2018/2019
528,Fulham FC,Timothy Fosu-Mensah,20,Right-Back,Manchester United,Loan,in,Summer,0.00,Premier League,2018,2018/2019


Как видим в 'en_18_sum' стало на три игрока меньше.

# Результат


Сделаем функцию для определения типа трансфера. 0 - обычный трансфер, 1 - свободный агент, 2 - аренда, 3 - возвращение из аренды.

In [ ]:
def transfer_type(fee):
  if fee[0] == 'E':
    return 3
  if fee[0] == "L":
    return 2
  if fee[0] == "F":
    return 1
  return 0

In [ ]:
for window in frames2:
  df = frames2[window]
  df['type'] = df.fee.apply(transfer_type)
  frames2[window] = df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Объеденим все трансферные окна и посчитаем число трансферов каждого вида.

In [ ]:
Transfers = pd.concat(frames2)

In [ ]:
Transfers['type'].value_counts()

0    929
2    493
3    301
1    143
Name: type, dtype: int64

In [ ]:
Transfers.position.value_counts()

Centre-Forward        315
Centre-Back           298
Central Midfield      242
Goalkeeper            160
Right Winger          152
Right-Back            143
Defensive Midfield    135
Left-Back             133
Left Winger           128
Attacking Midfield    100
Left Midfield          29
Right Midfield         19
Second Striker         11
midfield                1
Name: position, dtype: int64

In [ ]:
Transfers.to_csv('Transfers.csv')